In [1]:
# !pip install loguru
# !pip install nltk

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.model_selection import GridSearchCV
from loguru import logger as lg
import multiprocessing
import nltk
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import warnings
# Download necessary NLTK corpora
# nltk.download('stopwords')

In [3]:
warnings.filterwarnings("ignore")

In [28]:
NUM_PROCESSORS = (multiprocessing.cpu_count() - 2)
VERBOSE = 0
DEPARTMENTS_QUANTITY_TO_TRAIN = 6
lg.add("finnlp_binary_undersampling_experiment_1.log", rotation="1024 MB")
lg.info("\n\n####################################### NEW EXECUTION #######################################\n\n")
lg.info(f"Processors (cores) available: {NUM_PROCESSORS}")

2024-02-29 21:41:08.253 | INFO     | __main__:<module>:6 - 

####################################### NEW EXECUTION #######################################


2024-02-29 21:41:08.256 | INFO     | __main__:<module>:7 - Processors (cores) available: 40


In [5]:
brc = pd.read_csv("Banking_Regulation_Corpora_BRC_anonymous.csv" ,encoding="utf-8", low_memory=False, sep=";")

In [6]:
brc.head()

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
0,1,DPT_25,2020-12-23,2128,36560,2020-12-23,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,656,67.0,127.0,COMUNICADO,771984,https://www.bcb.gov.br/estabilidadefinanceira/...
1,1,DPT_25,2020-12-28,2129,36569,2020-12-28,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,2110,156.0,322.0,COMUNICADO,772973,https://www.bcb.gov.br/estabilidadefinanceira/...
2,1,DPT_25,2020-12-22,2130,36554,2020-12-22,BACEN,Divulga comunicado do Grupo de Ação Financeira...,120,16.0,18.0,Comunicamos que o Grupo de Ação Financeira con...,792,76.0,127.0,COMUNICADO,771452,https://www.bcb.gov.br/estabilidadefinanceira/...
3,1,DPT_25,2020-12-23,2131,4880,2020-12-23,BACEN,Dispõe sobre o horário de atendimento ao públi...,222,32.0,34.0,"O Banco Central do Brasil, na forma do art. 9º...",4964,365.0,875.0,RESOLUÇÃO CMN,772051,https://www.bcb.gov.br/estabilidadefinanceira/...
4,1,DPT_25,2020-12-23,2132,1125/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,165,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,3382,304.0,492.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771966,https://www.bcb.gov.br/estabilidadefinanceira/...


In [7]:
data_irrelevant = brc[brc['class'] == 0]

In [8]:
data_irrelevant.head(3)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
7846,0,DPT_25,2020-08-31,2169,0301/2020,2020-08-31,BACEN,Caracterizado fornecimento intempestivo ao Ba...,294,36.0,49.0,​DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONAD...,758,81.0,113.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729080,https://www.bcb.gov.br/estabilidadefinanceira/...
7847,0,DPT_25,2020-08-31,2170,36121,2020-08-31,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,7080,407.0,966.0,COMUNICADO,729131,https://www.bcb.gov.br/estabilidadefinanceira/...
7848,0,DPT_25,2020-08-31,2171,36123,2020-08-31,BACEN,Divulga nome de pessoas com intenção de ocupar...,140,19.0,21.0,Divulgamos os nomes de pretendentes a cargos d...,1558,149.0,276.0,COMUNICADO,729151,https://www.bcb.gov.br/estabilidadefinanceira/...


In [9]:
data_relevant = brc[brc['class'] == 1]

In [10]:
data_relevant.count()

class                   7846
department              7846
entry_date              7846
general_id              7846
normative_identifier    7846
publication_date        7846
regulatory_authority    7846
subject                 7846
subject_length          7846
subject_unique_words    7846
subject_words           7846
text                    7846
text_length             7846
text_unique_words       7846
text_words              7846
type                    7846
unique_document_id      7846
url                     7846
dtype: int64

In [11]:
data_relevant = data_relevant[data_relevant.text_words >= 50]
data_relevant.count()

class                   7714
department              7714
entry_date              7714
general_id              7714
normative_identifier    7714
publication_date        7714
regulatory_authority    7714
subject                 7714
subject_length          7714
subject_unique_words    7714
subject_words           7714
text                    7714
text_length             7714
text_unique_words       7714
text_words              7714
type                    7714
unique_document_id      7714
url                     7714
dtype: int64

In [12]:
data_irrelevant.count()

class                   54018
department              54018
entry_date              54018
general_id              54018
normative_identifier    54018
publication_date        54018
regulatory_authority    54018
subject                 54018
subject_length          54018
subject_unique_words    54018
subject_words           54018
text                    54018
text_length             54018
text_unique_words       54018
text_words              54018
type                    54018
unique_document_id      54018
url                     54018
dtype: int64

In [13]:
data_irrelevant = data_irrelevant[data_irrelevant.text_words >= 50]
data_irrelevant.count()

class                   53122
department              53122
entry_date              53122
general_id              53122
normative_identifier    53122
publication_date        53122
regulatory_authority    53122
subject                 53122
subject_length          53122
subject_unique_words    53122
subject_words           53122
text                    53122
text_length             53122
text_unique_words       53122
text_words              53122
type                    53122
unique_document_id      53122
url                     53122
dtype: int64

In [14]:
data_relevant['department'].unique()

array(['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16', 'DPT_24',
       'DPT_18', 'DPT_12', 'DPT_11', 'DPT_14', 'DPT_4', 'DPT_8', 'DPT_21',
       'DPT_1', 'DPT_15', 'DPT_20', 'DPT_22', 'DPT_10', 'DPT_23', 'DPT_7',
       'DPT_13', 'DPT_17', 'DPT_19', 'DPT_6'], dtype=object)

In [15]:
len(list(data_relevant['department'].unique()))

25

In [16]:
stop_words = set(stopwords.words('portuguese'))
df_stopwords = pd.DataFrame(list(stop_words), columns =['words'])
df_stopwords.head(3)

,words
0,estivemos
1,ao
2,terão


In [17]:
def remove_special_characters(df,column):
    df.loc[:,column] = df[column].str.lower()
    
    df.loc[:,column] = df[column].str.replace('á','a')
    df.loc[:,column] = df[column].str.replace('é','e')
    df.loc[:,column] = df[column].str.replace('í','i')
    df.loc[:,column] = df[column].str.replace('ó','o')
    df.loc[:,column] = df[column].str.replace('ú','u')

    df.loc[:,column] = df[column].str.replace('â','a')
    df.loc[:,column] = df[column].str.replace('ê','e')
    df.loc[:,column] = df[column].str.replace('î','i')
    df.loc[:,column] = df[column].str.replace('ô','o')
    df.loc[:,column] = df[column].str.replace('û','u')

    df.loc[:,column] = df[column].str.replace('à','a')
    df.loc[:,column] = df[column].str.replace('è','e')
    df.loc[:,column] = df[column].str.replace('ì','i')
    df.loc[:,column] = df[column].str.replace('ò','o')
    df.loc[:,column] = df[column].str.replace('ù','u')

    df.loc[:,column] = df[column].str.replace('ä','a')
    df.loc[:,column] = df[column].str.replace('ë','e')
    df.loc[:,column] = df[column].str.replace('ï','i')
    df.loc[:,column] = df[column].str.replace('ö','o')
    df.loc[:,column] = df[column].str.replace('ü','u')

    df.loc[:,column] = df[column].str.replace('ã','a')
    df.loc[:,column] = df[column].str.replace('ẽ','e')
    df.loc[:,column] = df[column].str.replace('ĩ','i')
    df.loc[:,column] = df[column].str.replace('õ','o')
    df.loc[:,column] = df[column].str.replace('ũ','u')

    df.loc[:,column] = df[column].str.replace('ç','c')
    
    return df

In [18]:
df_stopwords = remove_special_characters(df_stopwords,'words')

In [19]:
df_stopwords.head(3)

,words
0,estivemos
1,ao
2,terao


In [20]:
stop_words = df_stopwords['words'].tolist()

In [21]:
def clean_text(text, stop_words):
    text = [token for token in text.split(" ")]
    text = [word for word in text if not word in stop_words] # remove stopwords
    text = " ".join(text)
    return text

In [22]:
def cleaning(df,column, stop_words):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    # Remove stopwords
    df.loc[:,column] = df[column].apply(lambda x: clean_text(x, stop_words))

    return df

In [23]:
def remove_duplicates(sentence):
	words = sentence.split(" ")
	result = []
	for word in words:
		if word not in result:
			result.append(word)
	return result

<h3>Selecting the Departments</h3>

In [24]:
boards = data_relevant['department']
# Getting the 6 most populated boards
boards_list = list(boards.unique())[:DEPARTMENTS_QUANTITY_TO_TRAIN]
lg.info(f"boards_list: {boards_list}")

2024-02-29 21:36:31.851 | INFO     | __main__:<module>:4 - boards_list: ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']


<h3>Parameters</h3>

In [25]:
from sklearn.metrics import make_scorer
import sklearn.metrics as sk

METRICS = {
           'accuracy': make_scorer(sk.accuracy_score),
           'precision': make_scorer(sk.precision_score, average = 'weighted', zero_division=0),
           'recall': make_scorer(sk.recall_score, average = 'weighted', zero_division=0),
           'f1': make_scorer(sk.f1_score, average = 'binary', zero_division=0),
           'f1_macro': make_scorer(sk.f1_score, average = 'macro', zero_division=0),
           'f1_weighted': make_scorer(sk.f1_score, average = 'weighted', zero_division=0)
           }

CROSS_VALIDATION = 10
RAND_STATE = 42
REFIT_VAL = 'f1'

cls = dict({
        'random_forest': {
            'estimator': RandomForestClassifier(random_state=RAND_STATE, n_jobs = NUM_PROCESSORS),
            'parameters':{'n_estimators': [100, 400, 1000], 'max_depth': [10,30, 100], 'criterion':['entropy', 'log_loss', 'gini']}
            },
        'svm': {
            'estimator': SVC(random_state=RAND_STATE),
            'parameters':{'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear','poly','rbf','sigmoid']}
            },
        'xgboost':{
            'estimator': XGBClassifier(random_state=RAND_STATE, num_class=2, verbosity = 0, silent=True, n_jobs=NUM_PROCESSORS),
            'parameters':{'objective':['reg:squarederror','binary:logistic','multi:softmax','binary:hinge'],'n_estimators': [100,1000],'max_depth': [10,30],'learning_rate':[0.01,0.5]}
        },
        'nb':{
                'estimator': MultinomialNB(),
                'parameters':{'alpha':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001],'force_alpha': [True, False],'fit_prior': [True, False]}
            }
        
    })

In [26]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

def training_each_inductor_holdout(classifiers, board, X, y):
    for key in list(classifiers.keys()):
        lg.info(f"\n\n####### {key} INDUCTOR EXECUTING ####### - {board}\n\n")

        lg.debug(f"[+] classifiers[key]['estimator']: {classifiers[key]['estimator']} - {key} - {board}")
        lg.debug(f"[+] classifiers[key]['parameters']: {classifiers[key]['parameters']} - {key} - {board}")

        search = GridSearchCV(estimator=classifiers[key]['estimator'],
                            param_grid=classifiers[key]['parameters'],
                            cv = CROSS_VALIDATION,
                            scoring = METRICS,
                            refit = REFIT_VAL,
                            n_jobs = NUM_PROCESSORS,
                            verbose = VERBOSE)

        lg.debug(f"[+] Start GridSearchCV - {key} - {board}")
        search.fit(X, y)
        lg.debug(f"[+] End GridSearchCV - {key} - {board}")

        lg.info(f"search.best_estimator_: {search.best_estimator_} \n")

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=RAND_STATE, stratify=y)

        clf = search.best_estimator_.fit(x_train, y_train)
        
        y_pred = clf.predict(x_test)

        lg.success(f"\n\n####### METRICS {key} - {board} #######\n\n")

        lg.success(f"{key} - {board} - test_precision: {metrics.precision_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_recall: {metrics.recall_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_f1_binary: {metrics.f1_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_f1_macro: {metrics.f1_score(y_test, y_pred, average='macro')}")

<h3>The Training and Evaluation</h3>

In [ ]:
for board in boards_list:
    lg.info(f"\n\n####### {board} EXECUTING #######\n\n")

    total_relevant_documents_of_the_board = data_relevant[data_relevant.department == board].shape[0]
    
    ######################################################################
    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###

    # total_relevant_documents_of_the_board = int(data_relevant[data_relevant.department == board].shape[0]/10)

    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###
    ######################################################################

    lg.info(f"total_relevant_documents_of_the_board: {total_relevant_documents_of_the_board} - {board}")
    lg.info(f"Total irrelevant documents of the department: {data_irrelevant[data_irrelevant.department == board].shape[0]}")

    data_relevant_board = data_relevant[data_relevant.department == board]
    lg.info(f"data_relevant_board: {data_relevant_board.shape} - {board}")

    ####################################################################################################
    ############## TO TRAIN WITH A BALANCED DATASET (UNDERSAMPLING) OR IMBALANCED DATASET ##############
    # The total documents of the relevant class (total_relevant_documents_of_the_board) is use to get the first documents in the irrelevant class.

    data_irrelevant_board = data_irrelevant[data_irrelevant.department == board][:total_relevant_documents_of_the_board]
    # data_irrelevant_board = data_irrelevant[data_irrelevant.department == board]

    ############## TO TRAIN WITH A BALANCED DATASET (UNDERSAMPLING) OR IMBALANCED DATASET ##############
    ####################################################################################################

    ######################################################################
    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###

    # data_relevant_board = data_relevant[data_relevant.department == board][:total_relevant_documents_of_the_board]
    
    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###
    ######################################################################

    # data_irrelevant_board = data_irrelevant[data_irrelevant.department == board]
    lg.info(f"data_irrelevant_board: {data_irrelevant_board.shape} - {board}")

    df_gather_datasets = []
    df_gather_datasets.append(data_relevant_board)
    df_gather_datasets.append(data_irrelevant_board)
    # Merging all dataframes in the list as one dataframe.
    df_final_dataset = pd.concat(df_gather_datasets)
    
    lg.info(f"df_final_dataset: {df_final_dataset.shape} - {board}")

    # Cleaning the text and title
    lg.debug("[+] Start cleaning text and title")
    df_final_dataset = remove_special_characters(df_final_dataset, 'text')
    df_final_dataset = remove_special_characters(df_final_dataset, 'subject')
    df_final_dataset = cleaning(df_final_dataset, 'text', stop_words)
    df_final_dataset = cleaning(df_final_dataset, 'subject', stop_words)
    lg.debug("[+] End cleaning text and title")

    # To get only phrases with at lest three words after the preprocessing.
    df_final_dataset = df_final_dataset[df_final_dataset['text'].str.len() >= 3]

    data_x = df_final_dataset.text
    lg.debug("[+] Start TF-IDF")
    
    feature_extraction = TfidfVectorizer()
    data_x = feature_extraction.fit_transform(data_x)
    lg.debug("[+] End TF-IDF")

    data_y = df_final_dataset['class']

    training_each_inductor_holdout(cls, board, data_x, data_y)

    lg.success("\n\n[+][+][+] SUCCESS! [+][+][+]\n\n")

<h2>The output result is in the next cell. It was copied from the log file.</h2>

In [ ]:
# 2024-02-14 16:17:07.786 | INFO     | __main__:<module>:5 - 

# ####################################### NEW EXECUTION #######################################


# 2024-02-14 16:17:07.787 | INFO     | __main__:<module>:6 - Processors (cores) available: 38
# 2024-02-14 16:17:18.469 | INFO     | __main__:<module>:4 - boards_list: ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']
# 2024-02-14 16:17:18.502 | INFO     | __main__:<module>:2 - 

# ####### DPT_25 EXECUTING #######


# 2024-02-14 16:17:18.505 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 819 - DPT_25
# 2024-02-14 16:17:18.511 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 4051
# 2024-02-14 16:17:18.513 | INFO     | __main__:<module>:18 - data_relevant_board: (819, 18) - DPT_25
# 2024-02-14 16:17:18.518 | INFO     | __main__:<module>:39 - data_irrelevant_board: (819, 18) - DPT_25
# 2024-02-14 16:17:18.522 | INFO     | __main__:<module>:47 - df_final_dataset: (1638, 18) - DPT_25
# 2024-02-14 16:17:18.522 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-02-14 16:17:27.096 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-02-14 16:17:27.100 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-02-14 16:17:28.110 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-02-14 16:17:28.112 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_25


# 2024-02-14 16:17:28.114 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_25
# 2024-02-14 16:17:28.115 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_25
# 2024-02-14 16:17:28.115 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_25
# 2024-02-14 16:19:25.906 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_25
# 2024-02-14 16:19:25.909 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(max_depth=100, n_jobs=38, random_state=42) 

# 2024-02-14 16:19:26.310 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_25 #######


# 2024-02-14 16:19:26.317 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_25 - test_precision: 0.9724409448818898
# 2024-02-14 16:19:26.320 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_25 - test_recall: 0.7530487804878049
# 2024-02-14 16:19:26.324 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_25 - test_f1_binary: 0.8487972508591065
# 2024-02-14 16:19:26.327 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_25 - test_f1_macro: 0.8641246528268136
# 2024-02-14 16:19:26.327 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_25


# 2024-02-14 16:19:26.329 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_25
# 2024-02-14 16:19:26.330 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_25
# 2024-02-14 16:19:26.330 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_25
# 2024-02-14 16:20:28.234 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_25
# 2024-02-14 16:20:28.236 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=100.0, kernel='linear', random_state=42) 

# 2024-02-14 16:20:30.808 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_25 #######


# 2024-02-14 16:20:30.812 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_25 - test_precision: 0.9075907590759076
# 2024-02-14 16:20:30.816 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_25 - test_recall: 0.8384146341463414
# 2024-02-14 16:20:30.819 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_25 - test_f1_binary: 0.8716323296354992
# 2024-02-14 16:20:30.822 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_25 - test_f1_macro: 0.876344799178983
# 2024-02-14 16:20:30.823 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_25


# 2024-02-14 16:20:30.825 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_25
# 2024-02-14 16:20:30.826 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_25
# 2024-02-14 16:20:30.827 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_25
# 2024-02-14 21:32:54.679 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_25
# 2024-02-14 21:32:54.683 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.5, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=10, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=100, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-02-14 21:33:00.713 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_25 #######


# 2024-02-14 21:33:00.718 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_25 - test_precision: 0.9543859649122807
# 2024-02-14 21:33:00.721 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_25 - test_recall: 0.8292682926829268
# 2024-02-14 21:33:00.724 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_25 - test_f1_binary: 0.8874388254486134
# 2024-02-14 21:33:00.727 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_25 - test_f1_macro: 0.894363189548341
# 2024-02-14 21:33:00.728 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_25


# 2024-02-14 21:33:00.729 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_25
# 2024-02-14 21:33:00.730 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_25
# 2024-02-14 21:33:00.731 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_25
# 2024-02-14 21:33:02.728 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_25
# 2024-02-14 21:33:02.731 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=1e-05, force_alpha=True) 

# 2024-02-14 21:33:02.745 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_25 #######


# 2024-02-14 21:33:02.750 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_25 - test_precision: 0.9341085271317829
# 2024-02-14 21:33:02.754 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_25 - test_recall: 0.7347560975609756
# 2024-02-14 21:33:02.758 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_25 - test_f1_binary: 0.8225255972696245
# 2024-02-14 21:33:02.763 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_25 - test_f1_macro: 0.8396374542821952
# 2024-02-14 21:33:02.764 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-02-14 21:33:02.765 | INFO     | __main__:<module>:2 - 

# ####### DPT_3 EXECUTING #######


# 2024-02-14 21:33:02.769 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 688 - DPT_3
# 2024-02-14 21:33:02.777 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 1790
# 2024-02-14 21:33:02.780 | INFO     | __main__:<module>:18 - data_relevant_board: (688, 18) - DPT_3
# 2024-02-14 21:33:02.785 | INFO     | __main__:<module>:39 - data_irrelevant_board: (688, 18) - DPT_3
# 2024-02-14 21:33:02.796 | INFO     | __main__:<module>:47 - df_final_dataset: (1376, 18) - DPT_3
# 2024-02-14 21:33:02.798 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-02-14 21:33:37.290 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-02-14 21:33:37.294 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-02-14 21:33:41.450 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-02-14 21:33:41.452 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_3


# 2024-02-14 21:33:41.454 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_3
# 2024-02-14 21:33:41.455 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_3
# 2024-02-14 21:33:41.456 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_3
# 2024-02-14 21:36:33.436 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_3
# 2024-02-14 21:36:33.440 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(criterion='entropy', max_depth=100, n_estimators=400,
#                        n_jobs=38, random_state=42) 

# 2024-02-14 21:36:35.560 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_3 #######


# 2024-02-14 21:36:35.570 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_3 - test_precision: 0.8170731707317073
# 2024-02-14 21:36:35.575 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_3 - test_recall: 0.730909090909091
# 2024-02-14 21:36:35.579 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_3 - test_f1_binary: 0.7715930902111324
# 2024-02-14 21:36:35.583 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_3 - test_f1_macro: 0.7833869065513495
# 2024-02-14 21:36:35.584 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_3


# 2024-02-14 21:36:35.585 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_3
# 2024-02-14 21:36:35.586 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_3
# 2024-02-14 21:36:35.587 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_3
# 2024-02-14 21:37:57.403 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_3
# 2024-02-14 21:37:57.405 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=100.0, random_state=42) 

# 2024-02-14 21:38:00.395 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_3 #######


# 2024-02-14 21:38:00.403 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_3 - test_precision: 0.7977099236641222
# 2024-02-14 21:38:00.406 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_3 - test_recall: 0.76
# 2024-02-14 21:38:00.409 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_3 - test_f1_binary: 0.7783985102420857
# 2024-02-14 21:38:00.412 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_3 - test_f1_macro: 0.7838895206077685
# 2024-02-14 21:38:00.413 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_3


# 2024-02-14 21:38:00.415 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_3
# 2024-02-14 21:38:00.416 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_3
# 2024-02-14 21:38:00.417 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_3
# 2024-02-15 04:19:02.448 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_3
# 2024-02-15 04:19:02.453 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.01, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=10, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=1000, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-02-15 04:22:16.528 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_3 #######


# 2024-02-15 04:22:16.533 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_3 - test_precision: 0.7977099236641222
# 2024-02-15 04:22:16.536 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_3 - test_recall: 0.76
# 2024-02-15 04:22:16.539 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_3 - test_f1_binary: 0.7783985102420857
# 2024-02-15 04:22:16.542 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_3 - test_f1_macro: 0.7838895206077685
# 2024-02-15 04:22:16.543 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_3


# 2024-02-15 04:22:16.543 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_3
# 2024-02-15 04:22:16.544 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_3
# 2024-02-15 04:22:16.545 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_3
# 2024-02-15 04:22:19.048 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_3
# 2024-02-15 04:22:19.050 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=1e-05, force_alpha=True) 

# 2024-02-15 04:22:19.075 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_3 #######


# 2024-02-15 04:22:19.080 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_3 - test_precision: 0.6727941176470589
# 2024-02-15 04:22:19.083 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_3 - test_recall: 0.6654545454545454
# 2024-02-15 04:22:19.087 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_3 - test_f1_binary: 0.6691042047531992
# 2024-02-15 04:22:19.090 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_3 - test_f1_macro: 0.6714890393135365
# 2024-02-15 04:22:19.091 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-02-15 04:22:19.092 | INFO     | __main__:<module>:2 - 

# ####### DPT_2 EXECUTING #######


# 2024-02-15 04:22:19.096 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 683 - DPT_2
# 2024-02-15 04:22:19.110 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 5234
# 2024-02-15 04:22:19.113 | INFO     | __main__:<module>:18 - data_relevant_board: (683, 18) - DPT_2
# 2024-02-15 04:22:19.120 | INFO     | __main__:<module>:39 - data_irrelevant_board: (683, 18) - DPT_2
# 2024-02-15 04:22:19.130 | INFO     | __main__:<module>:47 - df_final_dataset: (1366, 18) - DPT_2
# 2024-02-15 04:22:19.131 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-02-15 04:22:39.277 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-02-15 04:22:39.282 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-02-15 04:22:41.779 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-02-15 04:22:41.781 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_2


# 2024-02-15 04:22:41.783 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_2
# 2024-02-15 04:22:41.784 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_2
# 2024-02-15 04:22:41.785 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_2
# 2024-02-15 04:24:49.608 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_2
# 2024-02-15 04:24:49.610 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(max_depth=100, n_estimators=1000, n_jobs=38,
#                        random_state=42) 

# 2024-02-15 04:24:53.574 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_2 #######


# 2024-02-15 04:24:53.581 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_2 - test_precision: 0.9527896995708155
# 2024-02-15 04:24:53.586 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_2 - test_recall: 0.8131868131868132
# 2024-02-15 04:24:53.589 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_2 - test_f1_binary: 0.8774703557312253
# 2024-02-15 04:24:53.592 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_2 - test_f1_macro: 0.8860140894302385
# 2024-02-15 04:24:53.593 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_2


# 2024-02-15 04:24:53.594 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_2
# 2024-02-15 04:24:53.595 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_2
# 2024-02-15 04:24:53.596 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_2
# 2024-02-15 04:25:51.480 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_2
# 2024-02-15 04:25:51.482 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=10.0, random_state=42) 

# 2024-02-15 04:25:54.023 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_2 #######


# 2024-02-15 04:25:54.027 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_2 - test_precision: 0.9444444444444444
# 2024-02-15 04:25:54.030 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_2 - test_recall: 0.8095238095238095
# 2024-02-15 04:25:54.034 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_2 - test_f1_binary: 0.8717948717948718
# 2024-02-15 04:25:54.037 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_2 - test_f1_macro: 0.8805311667321889
# 2024-02-15 04:25:54.037 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_2


# 2024-02-15 04:25:54.039 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_2
# 2024-02-15 04:25:54.040 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_2
# 2024-02-15 04:25:54.041 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_2
# 2024-02-15 09:36:59.686 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_2
# 2024-02-15 09:36:59.689 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.5, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=10, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=1000, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-02-15 09:37:21.198 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_2 #######


# 2024-02-15 09:37:21.203 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_2 - test_precision: 0.9537815126050421
# 2024-02-15 09:37:21.206 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_2 - test_recall: 0.8315018315018315
# 2024-02-15 09:37:21.209 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_2 - test_f1_binary: 0.888454011741683
# 2024-02-15 09:37:21.211 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_2 - test_f1_macro: 0.8953419286838775
# 2024-02-15 09:37:21.212 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_2


# 2024-02-15 09:37:21.219 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_2
# 2024-02-15 09:37:21.220 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_2
# 2024-02-15 09:37:21.222 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_2
# 2024-02-15 09:37:23.274 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_2
# 2024-02-15 09:37:23.277 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=1e-05, force_alpha=True) 

# 2024-02-15 09:37:23.296 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_2 #######


# 2024-02-15 09:37:23.300 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_2 - test_precision: 0.9159663865546218
# 2024-02-15 09:37:23.303 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_2 - test_recall: 0.7985347985347986
# 2024-02-15 09:37:23.306 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_2 - test_f1_binary: 0.8532289628180039
# 2024-02-15 09:37:23.310 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_2 - test_f1_macro: 0.8622920114261545
# 2024-02-15 09:37:23.311 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-02-15 09:37:23.312 | INFO     | __main__:<module>:2 - 

# ####### DPT_9 EXECUTING #######


# 2024-02-15 09:37:23.315 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 600 - DPT_9
# 2024-02-15 09:37:23.325 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 3619
# 2024-02-15 09:37:23.327 | INFO     | __main__:<module>:18 - data_relevant_board: (600, 18) - DPT_9
# 2024-02-15 09:37:23.334 | INFO     | __main__:<module>:39 - data_irrelevant_board: (600, 18) - DPT_9
# 2024-02-15 09:37:23.347 | INFO     | __main__:<module>:47 - df_final_dataset: (1200, 18) - DPT_9
# 2024-02-15 09:37:23.348 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-02-15 09:37:29.594 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-02-15 09:37:29.599 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-02-15 09:37:30.283 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-02-15 09:37:30.285 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_9


# 2024-02-15 09:37:30.286 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_9
# 2024-02-15 09:37:30.287 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_9
# 2024-02-15 09:37:30.287 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_9
# 2024-02-15 09:40:14.398 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_9
# 2024-02-15 09:40:14.402 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(criterion='entropy', max_depth=10, n_jobs=38,
#                        random_state=42) 

# 2024-02-15 09:40:14.823 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_9 #######


# 2024-02-15 09:40:14.829 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_9 - test_precision: 0.7952218430034129
# 2024-02-15 09:40:14.834 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_9 - test_recall: 0.9708333333333333
# 2024-02-15 09:40:14.838 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_9 - test_f1_binary: 0.874296435272045
# 2024-02-15 09:40:14.840 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_9 - test_f1_macro: 0.8586938850833293
# 2024-02-15 09:40:14.841 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_9


# 2024-02-15 09:40:14.843 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_9
# 2024-02-15 09:40:14.844 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_9
# 2024-02-15 09:40:14.845 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_9
# 2024-02-15 09:40:37.572 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_9
# 2024-02-15 09:40:37.574 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=0.8, kernel='sigmoid', random_state=42) 

# 2024-02-15 09:40:38.251 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_9 #######


# 2024-02-15 09:40:38.256 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_9 - test_precision: 0.8438661710037175
# 2024-02-15 09:40:38.260 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_9 - test_recall: 0.9458333333333333
# 2024-02-15 09:40:38.264 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_9 - test_f1_binary: 0.8919449901768172
# 2024-02-15 09:40:38.267 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_9 - test_f1_macro: 0.884996885332311
# 2024-02-15 09:40:38.269 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_9


# 2024-02-15 09:40:38.270 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_9
# 2024-02-15 09:40:38.272 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_9
# 2024-02-15 09:40:38.273 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_9
# 2024-02-15 16:33:17.009 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_9
# 2024-02-15 16:33:17.012 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.01, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=30, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=1000, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-02-15 16:34:11.449 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_9 #######


# 2024-02-15 16:34:11.454 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_9 - test_precision: 0.8410852713178295
# 2024-02-15 16:34:11.458 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_9 - test_recall: 0.9041666666666667
# 2024-02-15 16:34:11.461 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_9 - test_f1_binary: 0.8714859437751005
# 2024-02-15 16:34:11.464 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_9 - test_f1_macro: 0.8664789026234809
# 2024-02-15 16:34:11.464 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_9


# 2024-02-15 16:34:11.465 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_9
# 2024-02-15 16:34:11.466 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_9
# 2024-02-15 16:34:11.467 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_9
# 2024-02-15 16:34:13.778 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_9
# 2024-02-15 16:34:13.781 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=1e-05, force_alpha=True) 

# 2024-02-15 16:34:13.800 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_9 #######


# 2024-02-15 16:34:13.805 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_9 - test_precision: 0.823943661971831
# 2024-02-15 16:34:13.809 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_9 - test_recall: 0.975
# 2024-02-15 16:34:13.812 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_9 - test_f1_binary: 0.8931297709923663
# 2024-02-15 16:34:13.815 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_9 - test_f1_macro: 0.8823447020099446
# 2024-02-15 16:34:13.817 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-02-15 16:34:13.818 | INFO     | __main__:<module>:2 - 

# ####### DPT_5 EXECUTING #######


# 2024-02-15 16:34:13.822 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 534 - DPT_5
# 2024-02-15 16:34:13.831 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 2594
# 2024-02-15 16:34:13.833 | INFO     | __main__:<module>:18 - data_relevant_board: (534, 18) - DPT_5
# 2024-02-15 16:34:13.839 | INFO     | __main__:<module>:39 - data_irrelevant_board: (534, 18) - DPT_5
# 2024-02-15 16:34:13.854 | INFO     | __main__:<module>:47 - df_final_dataset: (1068, 18) - DPT_5
# 2024-02-15 16:34:13.856 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-02-15 16:34:23.528 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-02-15 16:34:23.532 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-02-15 16:34:24.777 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-02-15 16:34:24.778 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_5


# 2024-02-15 16:34:24.780 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_5
# 2024-02-15 16:34:24.781 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_5
# 2024-02-15 16:34:24.782 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_5
# 2024-02-15 16:36:15.522 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_5
# 2024-02-15 16:36:15.526 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(max_depth=10, n_estimators=400, n_jobs=38,
#                        random_state=42) 

# 2024-02-15 16:36:16.997 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_5 #######


# 2024-02-15 16:36:17.004 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_5 - test_precision: 0.7821011673151751
# 2024-02-15 16:36:17.010 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_5 - test_recall: 0.9392523364485982
# 2024-02-15 16:36:17.014 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_5 - test_f1_binary: 0.8535031847133758
# 2024-02-15 16:36:17.017 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_5 - test_f1_macro: 0.8371412027462983
# 2024-02-15 16:36:17.019 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_5


# 2024-02-15 16:36:17.020 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_5
# 2024-02-15 16:36:17.021 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_5
# 2024-02-15 16:36:17.022 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_5
# 2024-02-15 16:36:51.255 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_5
# 2024-02-15 16:36:51.258 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(kernel='poly', random_state=42) 

# 2024-02-15 16:36:52.772 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_5 #######


# 2024-02-15 16:36:52.776 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_5 - test_precision: 0.7619047619047619
# 2024-02-15 16:36:52.779 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_5 - test_recall: 0.9719626168224299
# 2024-02-15 16:36:52.782 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_5 - test_f1_binary: 0.8542094455852155
# 2024-02-15 16:36:52.785 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_5 - test_f1_macro: 0.8308987607329872
# 2024-02-15 16:36:52.786 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_5


# 2024-02-15 16:36:52.788 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_5
# 2024-02-15 16:36:52.789 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_5
# 2024-02-15 16:36:52.790 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_5
# 2024-02-15 20:57:51.127 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_5
# 2024-02-15 20:57:51.131 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.01, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=30, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=1000, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-02-15 20:59:12.959 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_5 #######


# 2024-02-15 20:59:12.964 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_5 - test_precision: 0.8214285714285714
# 2024-02-15 20:59:12.968 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_5 - test_recall: 0.8598130841121495
# 2024-02-15 20:59:12.971 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_5 - test_f1_binary: 0.8401826484018264
# 2024-02-15 20:59:12.974 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_5 - test_f1_macro: 0.8363592667846452
# 2024-02-15 20:59:12.974 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_5


# 2024-02-15 20:59:12.975 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_5
# 2024-02-15 20:59:12.976 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_5
# 2024-02-15 20:59:12.977 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_5
# 2024-02-15 20:59:15.198 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_5
# 2024-02-15 20:59:15.200 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=0.1, force_alpha=True) 

# 2024-02-15 20:59:15.218 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_5 #######


# 2024-02-15 20:59:15.222 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_5 - test_precision: 0.7795275590551181
# 2024-02-15 20:59:15.225 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_5 - test_recall: 0.9252336448598131
# 2024-02-15 20:59:15.228 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_5 - test_f1_binary: 0.8461538461538461
# 2024-02-15 20:59:15.230 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_5 - test_f1_macro: 0.8302934179222838
# 2024-02-15 20:59:15.231 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-02-15 20:59:15.232 | INFO     | __main__:<module>:2 - 

# ####### DPT_16 EXECUTING #######


# 2024-02-15 20:59:15.236 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 439 - DPT_16
# 2024-02-15 20:59:15.243 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 1558
# 2024-02-15 20:59:15.246 | INFO     | __main__:<module>:18 - data_relevant_board: (439, 18) - DPT_16
# 2024-02-15 20:59:15.251 | INFO     | __main__:<module>:39 - data_irrelevant_board: (439, 18) - DPT_16
# 2024-02-15 20:59:15.267 | INFO     | __main__:<module>:47 - df_final_dataset: (878, 18) - DPT_16
# 2024-02-15 20:59:15.268 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-02-15 20:59:29.305 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-02-15 20:59:29.309 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-02-15 20:59:31.059 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-02-15 20:59:31.061 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_16


# 2024-02-15 20:59:31.062 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_16
# 2024-02-15 20:59:31.064 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_16
# 2024-02-15 20:59:31.065 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_16
# 2024-02-15 21:01:27.374 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_16
# 2024-02-15 21:01:27.377 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(criterion='entropy', max_depth=100, n_estimators=400,
#                        n_jobs=38, random_state=42) 

# 2024-02-15 21:01:29.037 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_16 #######


# 2024-02-15 21:01:29.043 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_16 - test_precision: 0.84
# 2024-02-15 21:01:29.049 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_16 - test_recall: 0.9545454545454546
# 2024-02-15 21:01:29.052 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_16 - test_f1_binary: 0.8936170212765958
# 2024-02-15 21:01:29.055 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_16 - test_f1_macro: 0.8858329008822003
# 2024-02-15 21:01:29.056 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_16


# 2024-02-15 21:01:29.057 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_16
# 2024-02-15 21:01:29.058 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_16
# 2024-02-15 21:01:29.059 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_16
# 2024-02-15 21:01:47.910 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_16
# 2024-02-15 21:01:47.912 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=0.1, kernel='sigmoid', random_state=42) 

# 2024-02-15 21:01:48.694 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_16 #######


# 2024-02-15 21:01:48.698 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_16 - test_precision: 0.7510729613733905
# 2024-02-15 21:01:48.702 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_16 - test_recall: 0.9943181818181818
# 2024-02-15 21:01:48.705 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_16 - test_f1_binary: 0.8557457212713936
# 2024-02-15 21:01:48.708 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_16 - test_f1_macro: 0.8278728606356968
# 2024-02-15 21:01:48.709 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_16


# 2024-02-15 21:01:48.711 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_16
# 2024-02-15 21:01:48.712 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_16
# 2024-02-15 21:01:48.713 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_16
# 2024-02-16 01:21:22.733 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_16
# 2024-02-16 01:21:22.736 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.5, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=30, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=1000, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-02-16 01:21:35.289 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_16 #######


# 2024-02-16 01:21:35.293 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_16 - test_precision: 0.861878453038674
# 2024-02-16 01:21:35.297 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_16 - test_recall: 0.8863636363636364
# 2024-02-16 01:21:35.299 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_16 - test_f1_binary: 0.8739495798319329
# 2024-02-16 01:21:35.302 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_16 - test_f1_macro: 0.8721332913568887
# 2024-02-16 01:21:35.303 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_16


# 2024-02-16 01:21:35.304 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_16
# 2024-02-16 01:21:35.305 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_16
# 2024-02-16 01:21:35.306 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_16
# 2024-02-16 01:21:37.292 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_16
# 2024-02-16 01:21:37.294 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=1e-05, force_alpha=True) 

# 2024-02-16 01:21:37.309 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_16 #######


# 2024-02-16 01:21:37.312 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_16 - test_precision: 0.8088235294117647
# 2024-02-16 01:21:37.315 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_16 - test_recall: 0.9375
# 2024-02-16 01:21:37.318 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_16 - test_f1_binary: 0.868421052631579
# 2024-02-16 01:21:37.321 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_16 - test_f1_macro: 0.857050032488629
# 2024-02-16 01:21:37.322 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


